In [9]:
import pandas as pd
import requests
import json
import time
import matplotlib.pyplot as plt

In [2]:
addresses = ['2993tWVVBGnzkWzLWTSLHByX9FAaNH7j1QoQBnDgxH71','arsc4jbDnzaqcCLByyGo7fg7S2SmcFsWUzQuDtLZh2y','9J41nRFg5aega9WAcrSxR9NBJRneQ5sWptyv1cAcFG5j', 'FeQ6UuJXJoeVkBi6p7eNMxH2E962TktzSMo9kpuQnsgG', 'Habp5bncMSsBC3vkChyebepym5dcTNRYeg2LVG464E96' , 'HPnPbn8ryBpPhW9ddMo8BdDLm3ec6j9fes1JWCyhVtZA', 'HWEoBxYs7ssKuudEjzjmpfJVX7Dvi7wescFsVx2L5yoY',
            'MJKqp326RZCHnAAbew9MDdui3iCKWco7fsK9sVuZTX2']

In [3]:
magic_eden_activities_data = []

In [4]:
'''
    Fetch On-chain wallet NFT activities data from magic eden api  
'''

for address in addresses:
    time.sleep(1)
    
    url = f"https://api-mainnet.magiceden.dev/v2/wallets/{address}/activities"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)

    parsedTxResult = []
    if response.status_code == 200:
        result = response.json()
        
        for data in result:
            parsedTxResult.append(
                {
                    'blockTime': data.get('blockTime'),
                    'type': data.get('type'),
                    'priceInfo': data.get('priceInfo'),
                    'price': data.get('price'),
                    'collection': data.get('collection'),
                }
            )
        magic_eden_activities_data.append(parsedTxResult)
    else:
        print(f"Request failed with status code {response.status_code}")

In [5]:
magic_eden_activities_data

[[{'blockTime': 1716710152,
   'type': 'buyNow',
   'priceInfo': {'solPrice': {'rawAmount': '1810000000000000000',
     'address': 'So11111111111111111111111111111111111111112',
     'decimals': 9}},
   'price': 1.81,
   'collection': 'backwoods'},
  {'blockTime': 1716708723,
   'type': 'list',
   'priceInfo': {'solPrice': {'rawAmount': '1810000000000000000',
     'address': 'So11111111111111111111111111111111111111112',
     'decimals': 9}},
   'price': 1.81,
   'collection': 'backwoods'},
  {'blockTime': 1716708130,
   'type': 'list',
   'priceInfo': {'solPrice': {'rawAmount': '1830000000000000000',
     'address': 'So11111111111111111111111111111111111111112',
     'decimals': 9}},
   'price': 1.83,
   'collection': 'backwoods'},
  {'blockTime': 1716707088,
   'type': 'list',
   'priceInfo': {'solPrice': {'rawAmount': '1924000000000000000',
     'address': 'So11111111111111111111111111111111111111112',
     'decimals': 9}},
   'price': 1.924,
   'collection': 'backwoods'},
  {'block

In [6]:
features_list =[]

In [8]:
'''
    Algorithm to Categorize the wallet based on his NFT activities
'''

for address, activity_data in zip(addresses,magic_eden_activities_data):
    if len(activity_data) == 0:
        continue

    df = pd.DataFrame(activity_data)

    df['blockTime'] = pd.to_datetime(df['blockTime'], unit='s')

    '''
     In the below commented code, I tried to score a user based on their interactions and NFT volume using normilization 
     but ultimately failed . might be resolved in future by a better idea.
    '''
    
    # df['is_buy'] = df['type'].apply(lambda x: 1 if x == 'buyNow' else 0)
    # df['is_sell'] = df['type'].apply(lambda x: 1 if x == 'list' else 0)
    # df['is_mint'] = df['type'].apply(lambda x: 1 if x == 'mint' else 0)

    # num_transactions = len(df)
    # total_volume = df['price'].sum()
    # buy_sell_ratio = df['is_buy'].sum() / (df['is_sell'].sum() + 1)  
    # activity_diversity = df['type'].nunique()
    # collection_diversity = df['collection'].nunique()
    # time_activity = (df['blockTime'].max() - df['blockTime'].min()).days / num_transactions 

    # features = {
    #     'num_transactions': num_transactions,
    #     'total_volume': total_volume,
    #     'buy_sell_ratio': buy_sell_ratio,
    #     'activity_diversity': activity_diversity,
    #     'collection_diversity': collection_diversity,
    #     'time_activity': time_activity
    # }

    min_date = df['blockTime'].min()
    max_date = df['blockTime'].max()
    num_weeks = (max_date - min_date).days / 7
    
    avg_transactions_per_week = len(df) / num_weeks
    activity_span_in_months = (max_date - min_date).days / 30
    unique_transaction_types = df['type'].nunique()
    total_transactions = len(df)

    freq_criteria = avg_transactions_per_week >= 1
    consistency_criteria = activity_span_in_months >= 3
    diversity_criteria = unique_transaction_types >= 2
    volume_criteria = total_transactions >= 10

    is_regular = all([freq_criteria, consistency_criteria, diversity_criteria, volume_criteria])

    print(address ,"is this wallet a regular user?", is_regular)

    

2993tWVVBGnzkWzLWTSLHByX9FAaNH7j1QoQBnDgxH71 is this wallet a regular user? True
FeQ6UuJXJoeVkBi6p7eNMxH2E962TktzSMo9kpuQnsgG is this wallet a regular user? False
